In [13]:
from sfm.data.sci_data.SFMDecTokenizer import SFMDecTokenizer
import torch
import os
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from copy import deepcopy

In [14]:
def show_ckpt(name, ckpt):
    for k, v in ckpt.items():
        if 'dummy' not in k:
            print(name, k, v.shape)

def process_protein(output):
    if '</protein>' not in output:
        return None
    m = re.search(r'<protein>(.*?)</protein>', output)
    if m:
        s = m.group(1)
        s = s.replace('<a>', '')
        s = s.replace(' ', '')
        return s.strip()
    return None

In [15]:
tokenizer_home = '/hai1/mfm/ds_dataset/llama2/llama-2-7b'
tokenizer = SFMDecTokenizer.from_pretrained(
    tokenizer_home,
    prot_spm_path='/blob/shufxi/data/scigpt/ur50bpe/bpe',
    dna_spm_path='/blob/shufxi/data/scigpt/dnabpe/bpe',
    rna_spm_path='/blob/shufxi/data/scigpt/rnabpe/bpe',
)
print(len(tokenizer))
print(tokenizer.tokenize('<protein>AABBCCDD</protein>'))
llama_tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_home,
)
print(len(llama_tokenizer))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'SFMDecTokenizer'.


40014
['<protein>', '▁A', 'AB', 'B', 'CC', 'DD', '</protein>']
32000


In [16]:
model = AutoModelForCausalLM.from_pretrained(tokenizer_home)

Loading checkpoint shards: 100%|██████████| 2/2 [00:23<00:00, 11.64s/it]


In [23]:
# ckpt_home = '/hai1/shufxi/scigpt/7bv2/stageB/global_step26999/'
# ckpt_home = r'/home/yinxia/hai1/shufxi/scigpt/7bv3/stageB/global_step32999'
# ckpt_home = r'/home/yinxia/blob1.v2/shufxi/scigpt/7bv3/inst/20240227121523/global_step3585'
# ckpt_home = r'/home/yinxia//blob1.v2/shufxi/scigpt/7bv3/inst/20240301131100/global_step4753'
# ckpt_home = r'/home/yinxia/hai1/shufxi/scigpt/7bv3/stageB.prot/global_step224655'

# ckpt_home = r'/hai1/mfm/shufxi/scigpt/7bv3/stageA_200k/global_step140999/'
# ckpt_home = r'/hai1/mfm/shufxi/scigpt/7bv3/stageA_200k/global_step999/'
ckpt_home = r'/hai1/mfm/shufxi/scigpt/7bv3/stageA_prot_e10_bs256/global_step19999/' # full finetune
# ckpt_home = r'/hai1/mfm/shufxi/scigpt/7bv3/stageA_prot_e10_bs256_emb/global_step29999/' # emb finetune, but not load llama emb
# ckpt_home = r"/hai1/mfm/shufxi/scigpt/7bv3/stageA_prot_e10_bs256_emb_bugfix_a100/global_step4999/" # emb finetune, load llama emb

In [24]:
test_model = deepcopy(model)
model_dict = test_model.state_dict()
ckpt_dict = {}

layer0 = torch.load(os.path.join(ckpt_home, "layer_00-model_states.pt"), map_location=torch.device("cpu"))
ckpt_dict['model.embed_tokens.weight'] = layer0['embed_tokens.weight']#[:32000]
show_ckpt('layer0', layer0)
for l in range(0, 32):
    l_index = str(l + 1).zfill(2)
    layer = torch.load(os.path.join(ckpt_home, f"layer_{l_index}-model_states.pt"), map_location=torch.device("cpu"))
    show_ckpt(l_index, layer)
    for k in layer:
        if "dummy" in k or 'rotary_emb' in k:
            continue
        ckpt_dict[f"model.layers.{l}.{k}"] = layer[k]

layer = torch.load(os.path.join(ckpt_home, "layer_33-model_states.pt"), map_location=torch.device("cpu"))
show_ckpt(33, layer)
ckpt_dict["model.norm.weight"] = layer["norm.weight"]

layer = torch.load(os.path.join(ckpt_home, "layer_34-model_states.pt"), map_location=torch.device("cpu"))
show_ckpt(34, layer)
ckpt_dict["lm_head.weight"] = layer["lm_head.weight"]#[:32000]
model_dict.update(ckpt_dict)

test_model.resize_token_embeddings(len(tokenizer))
test_model.load_state_dict(model_dict)
test_model = test_model.cuda()
test_model.eval()

layer0 embed_tokens.weight torch.Size([40014, 4096])
01 self_attn.q_proj.weight torch.Size([4096, 4096])
01 self_attn.k_proj.weight torch.Size([4096, 4096])
01 self_attn.v_proj.weight torch.Size([4096, 4096])
01 self_attn.o_proj.weight torch.Size([4096, 4096])
01 mlp.gate_proj.weight torch.Size([11008, 4096])
01 mlp.up_proj.weight torch.Size([11008, 4096])
01 mlp.down_proj.weight torch.Size([4096, 11008])
01 input_layernorm.weight torch.Size([4096])
01 post_attention_layernorm.weight torch.Size([4096])
02 self_attn.q_proj.weight torch.Size([4096, 4096])
02 self_attn.k_proj.weight torch.Size([4096, 4096])
02 self_attn.v_proj.weight torch.Size([4096, 4096])
02 self_attn.o_proj.weight torch.Size([4096, 4096])
02 mlp.gate_proj.weight torch.Size([11008, 4096])
02 mlp.up_proj.weight torch.Size([11008, 4096])
02 mlp.down_proj.weight torch.Size([4096, 11008])
02 input_layernorm.weight torch.Size([4096])
02 post_attention_layernorm.weight torch.Size([4096])
03 self_attn.q_proj.weight torch.Size

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 79.14 GiB of which 8.06 MiB is free. Including non-PyTorch memory, this process has 78.99 GiB memory in use. Of the allocated memory 78.49 GiB is allocated by PyTorch, and 12.71 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [22]:
print(torch.sum(torch.abs(model.state_dict()['model.embed_tokens.weight'] - test_model.state_dict()['model.embed_tokens.weight'][:32000].cpu())))
print(torch.sum(torch.abs(model.state_dict()['model.layers.10.self_attn.k_proj.weight'] - test_model.state_dict()['model.layers.10.self_attn.k_proj.weight'].cpu())))
print(torch.sum(torch.abs(model.state_dict()['lm_head.weight'] - test_model.state_dict()['lm_head.weight'][:32000].cpu())))

tensor(0.0063)
tensor(6.8878e-05)
tensor(0.0007)


In [40]:
model_dict.update(ckpt_dict)
print(model_dict['model.embed_tokens.weight'].shape, model_dict['lm_head.weight'].shape)
emb=model.state_dict()['model.embed_tokens.weight']
lm_head=model.state_dict()['lm_head.weight']
print(emb.shape, lm_head.shape)
model_dict['model.embed_tokens.weight'][:emb.shape[0]] = emb[:]
#model_dict['lm_head.weight'][:lm_head.shape[0]] = lm_head[:]
# model_dict['model.embed_tokens.weight'] = emb[:]
# model_dict['lm_head.weight'] = lm_head[:]

#test_model.resize_token_embeddings(emb.shape[0])
#test_model.resize_token_embeddings(len(tokenizer))
test_model.load_state_dict(model_dict)

print(model_dict['model.embed_tokens.weight'].shape, model_dict['lm_head.weight'].shape)

torch.Size([40014, 4096]) torch.Size([40014, 4096])
torch.Size([32000, 4096]) torch.Size([32000, 4096])


torch.Size([40014, 4096]) torch.Size([40014, 4096])


In [9]:
print(model.state_dict()['model.embed_tokens.weight'].shape)
print(model.state_dict()['lm_head.weight'].shape)
print(ckpt_dict['model.embed_tokens.weight'].shape)
print(ckpt_dict['lm_head.weight'].shape)
print(test_model.state_dict()['model.embed_tokens.weight'].shape)
print(test_model.state_dict()['lm_head.weight'].shape)

torch.Size([32000, 4096])
torch.Size([32000, 4096])
torch.Size([40014, 4096])
torch.Size([40014, 4096])
torch.Size([40014, 4096])
torch.Size([40014, 4096])


In [12]:
#input_ids = llama_tokenizer('An apple a day', return_tensors="pt").input_ids.cuda()
input_ids = tokenizer('An apple a day', return_tensors="pt").input_ids.cuda()
input_ids = tokenizer('<protein>', return_tensors="pt").input_ids.cuda()
output = test_model.generate(
    input_ids,
    num_beams=4,
    max_new_tokens=100,
    num_return_sequences=4,
    return_dict_in_generate=True,
    # output_scores=True,
    do_sample=False,
    # repetition_penalty=1.2,
    # num_beams=5,
    # max_new_tokens=512,
    # num_return_sequences=1,
    # return_dict_in_generate=True,
    # output_scores=True,
    # do_sample=True,
    # top_p=0.95,
    # repetition_penalty=1.5,
)

res = tokenizer.decode(output.sequences[0], skip_special_tokens=False)
print(res)
# for i in range(len(output)):
#     s = llama_tokenizer.decode(output[i])
#     print(s)
#     # print(s, output.sequences_scores[i].item())
#     # s = tokenizer.decode(output.sequences[i])
#     # print(s, output.sequences_scores[i].item())

<s> <protein> I'm not sure what you're trying to do, but I'm pretty sure you're not trying to do it. I'm not sure what you're trying to do, but I'm pretty sure you're not trying to do it. I'm not sure what you're trying to do, but I'm pretty sure you're not trying to do it. I'm not sure what you're trying to do, but I'


Bad pipe message: %s [b'\t\xe0F%\xa9t\xdb\xbc}\x1b\x9f\x0b\\[V\xe6"y\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0\'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c']
Bad pipe message: %s [b'\xacM\xa8\xa0\xee\xdcB\x15\xe6\x84|Qu\xd9\x85(\xe8\xac\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0']
Bad pipe message: %s [b"a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0s\xc0w\x00\xc4\x00\xc3\xc0#\xc0'\x00g\x00@\xc0r\xc0v\x00\xbe\x00\xbd\xc0\n\xc0\x14\x009\x008\x00\

In [33]:
print(type(llama_tokenizer), type(tokenizer))
print(llama_tokenizer.tokenize('An apple  a day\nYes\nThis is\nan apple\n'))
print(tokenizer.tokenize('An apple  a day\nYes\nThis is\n\nan apple\n'))

<class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'> <class 'sfm.data.sci_data.SFMDecTokenizer.SFMDecTokenizer'>
['▁An', '▁apple', '▁', '▁a', '▁day', '<0x0A>', 'Yes', '<0x0A>', 'This', '▁is', '<0x0A>', 'an', '▁apple', '<0x0A>']
['▁An', '▁apple', '▁', '▁a', '▁day', '<0x0A>', 'Yes', '<0x0A>', 'This', '▁is', '<0x0A>', '<0x0A>', 'an', '▁apple']


In [18]:
from sfm.data.sci_data.dataset import ProcessedSciDataset
from sfm.data.sci_data.SFMDecTokenizer import SFMDecTokenizer
from sfm.logging import logger
from sfm.models.scigpt.config import (
    ScigptConfig,
    scigpt_7b_config,
    scigpt_13b_config,
    scigpt_350m_config,
    scigpt_shallow_config,
    scigpt_tiny_config,
)
from sfm.models.scigpt.scigpt import ScigptModel
from sfm.pipeline.accelerator.trainer import Trainer
from sfm.utils import arg_utils
from sfm.utils.cli_utils import cli

config_registry = {
    "scigpt_tiny": scigpt_tiny_config,
    "scigpt_shallow": scigpt_shallow_config,
    "scigpt_350m": scigpt_350m_config,
    "scigpt": scigpt_shallow_config,
    "scigpt_7b": scigpt_7b_config,
    "scigpt_13b": scigpt_13b_config,
}


from argparse import ArgumentParser
args = ArgumentParser()
args.model_type="scigpt_7b_config"
args.vocab_size=40014
args.pad_token_id=32000
args.max_position_embeddings=4096
args.bf16=True
args.strategy="Pipeline"
args.pipeline_model_parallel_size=1
args.pp_partition_layer_name="LlamaDecoderLayerPP"
args.load_ckpt=True
args.pretrained_ckpt_path="/hai1/mfm/ds_dataset/llama2/llama-2-7b/"
args.unfreeze_param_list="lm_head.weight,embed_tokens.weight"
args.learnable_cutoff=32000
args.infer=True
args.llm_model_name_or_path="/hai1/mfm/ds_dataset/llama2/llama-2-7b/"



config = arg_utils.from_args(args, ScigptConfig)
config = config_registry.get(config.model_type, scigpt_tiny_config)(config)
config.llm_model_name_or_path="/hai1/mfm/ds_dataset/llama2/llama-2-7b/"

model = ScigptModel(config)

In [9]:
tokenizer_home = '/hai1/mfm/ds_dataset/llama2/llama-2-7b'
tokenizer = SFMDecTokenizer.from_pretrained(
    tokenizer_home,
    prot_spm_path='/blob/shufxi/data/scigpt/ur50bpe/bpe',
    dna_spm_path='/blob/shufxi/data/scigpt/dnabpe/bpe',
    rna_spm_path='/blob/shufxi/data/scigpt/rnabpe/bpe',
)
print(len(tokenizer))
print(tokenizer.tokenize('<protein>AABBCCDD</protein>'))
print(tokenizer.tokenize('An apple  a day\nYes\nThis is\n\nan applenade\n'))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'SFMDecTokenizer'.


40014
['<protein>', '▁A', 'AB', 'B', 'CC', 'DD', '</protein>']
['▁An', '▁apple', '▁', '▁a', '▁day', '<0x0A>', 'Yes', '<0x0A>', 'This', '▁is', '<0x0A>', '<0x0A>', 'an', '▁app', 'len', 'ade']


In [12]:
model.eval()
model=model.cuda()

input_ids = tokenizer('An apple a day', return_tensors="pt").input_ids.cuda()
#input_ids = tokenizer('<protein>AA', return_tensors="pt").input_ids.cuda()
output = model.decoder.generate(
    input_ids,
    num_beams=4,
    max_new_tokens=100,
    num_return_sequences=4,
    return_dict_in_generate=True,
    do_sample=False,
)

res = tokenizer.decode(output.sequences[0], skip_special_tokens=False)
print(res)

AttributeError: 'ScigptModel' object has no attribute 'decoder'